# Generate a stream of images

In this first notebook we show how we generate a stream of image blocks. As a first example, we generate an horizontal stream of block images. 

Let $\mathcal{I}$ be a "big" image of size $n\times M$, with $M \gg n$, we generate a flow of smaller images $I_{t}$, of size $n\times n$, for $t=0, \ldots, \frac{M}{n}$.

To do so we are going to use an abstract class that we defined in our module called ```AbstractImageStreamingGenerator```.


In [ ]:
import numpy as np
from scipy.misc import imread
from matplotlib import pyplot as plt

# loadlib add to syspath SST modules
import loadlib

from SST.streaming.streaming_generators import AbstractImageStreamingGenerator

# plt.rcParams['figure.figsize'] = (12.0, 12.0)
%load_ext autoreload

%matplotlib inline

In [ ]:
%autoreload 2

In [ ]:
# loading an image test
test_img = imread('RGB_US-CA-SanDiego_2010_05_03.tif')
# we crop test_img to obtain initial big image
I = test_img[500:600,:]

print("Size of image I from which we create the stream is: ", I.shape[:2])
plt.figure(figsize=(20,4))
plt.imshow(I)
plt.show()

In order to generate our stream we are going to create a class ```HorizontalStream``` that inherits from the class ```AbstractImageStreamingGenerator```. 
The parent class has two main methods:
-  ```__init__(self, img)```, the constructor that takes as input the big image
-  ```generate_stream(self, block_shape, **kwargs)```, an abstract method that we have to implement each time we inherit from ```AbstractImageStreamingGenerator```.

The idea is to use this method to return a generator that simulate a streaming of block images.   

In [ ]:
class HorizontalStream(AbstractImageStreamingGenerator):
    def generate_stream(self, block_shape, **kwargs):
        b_nr, b_nc = block_shape # number of row and number of columns in each block
        nr, nc, nz = self.img.shape
        
        for i in range(0, nc, b_nc):
            # selecting at each iteration an image block of size (b_nr, b_nc)
            yield self.img[:b_nr, i:i+b_nc]

In [ ]:
## We test our generator of streaming
gen = HorizontalStream(I)
# we choose 100,100 as block shape only because I has 100 rows
stream = gen.generate_stream(block_shape=(100,100))

In [ ]:
# we plot all the images in our stream
for img in stream:
    plt.figure()
    plt.imshow(img)
    plt.axis('off')
    plt.show()

## Another example of streaming generator

In this way is possible to write any kind of stream generator, and make them return not only images but also other related information. For example, we can modify our ```HorizontalStream``` class to make it returns, along with the ith block image, also the $4$-connected graph associated.

In [ ]:
from scipy.sparse.csgraph import minimum_spanning_tree
from SST.utils import plot_graph, img_to_graph

In [ ]:
class HorizontalStream(AbstractImageStreamingGenerator):
    def generate_stream(self, block_shape, **kwargs):
        b_nr, b_nc = block_shape # number of row and number of columns in each block
        nr, nc, nz = self.img.shape
        
        for i in range(0, nc, b_nc):
            # selecting at each iteration an image block of size (b_nr, b_nc)
            img = self.img[:b_nr, i:i+b_nc]
            g = img_to_graph(img)
            yield img, g

In the following example we are going to plot, for each element of the stream, the block image and the minimum spanning tree of the $4$-connected graph associated to the ith image.

In [ ]:
gen = HorizontalStream(I)
stream = gen.generate_stream(block_shape=(100,100))

for n, (img,g) in enumerate(stream):
    if n > 3:
        # since is only an example we stop after 4 iterations
        break
    plot_graph(img, minimum_spanning_tree(g))
    
  